# Python Decorators

Viri:
- [Primer on Python Decorators](https://realpython.com/primer-on-python-decorators/)
- [Decorators in Python](https://www.datacamp.com/community/tutorials/decorators-python)
- [Reuven M. Lerner - Practical decorators - PyCon 2019](https://www.youtube.com/watch?v=MjHpMCIvwsY)

Decorators provide a simple syntax for calling higher-order functions.

By definition, a decorator is a function that takes another function and extends the behavior of the latter function without explicitly modifying it.

## Functions

### First-Class Objects

In Python, functions are first-class objects. This means that functions can be passed around and used as arguments, just like any other object (string, int, float, list, and so on). Consider the following three functions:

In [1]:
def say_hello(name):
    return f"Hello {name}"

def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def greet_bob(greeter_func):
    return greeter_func("Bob")

Here, say_hello() and be_awesome() are regular functions that expect a name given as a string. The greet_bob() function however, expects a function as its argument. We can, for instance, pass it the say_hello() or the be_awesome() function:

In [2]:
greet_bob(say_hello)

'Hello Bob'

In [3]:
greet_bob(be_awesome)

'Yo Bob, together we are the awesomest!'

Note that greet_bob(say_hello) refers to two functions, but in different ways: greet_bob() and say_hello. The say_hello function is named without parentheses. This means that only a reference to the function is passed. The function is not executed. The greet_bob() function, on the other hand, is written with parentheses, so it will be called as usual.

### Inner Functions

It’s possible to define functions inside other functions. Such functions are called inner functions. Here’s an example of a function with two inner functions:

In [4]:
def parent():
    print("Printing from the parent() function")

    def first_child():
        print("Printing from the first_child() function")

    def second_child():
        print("Printing from the second_child() function")

    second_child()
    first_child()

What happens when you call the parent() function? Think about this for a minute. The output will be as follows:

In [5]:
parent()

Printing from the parent() function
Printing from the second_child() function
Printing from the first_child() function


Note that the order in which the inner functions are defined does not matter. Like with any other functions, the printing only happens when the inner functions are executed.

Furthermore, the inner functions are not defined until the parent function is called. They are locally scoped to parent(): they only exist inside the parent() function as local variables. Try calling first_child(). You should get an error:

In [6]:
first_child()

NameError: name 'first_child' is not defined

Whenever you call parent(), the inner functions first_child() and second_child() are also called. But because of their local scope, they aren’t available outside of the parent() function.

### Returning Functions From Functions

Python also allows you to use functions as return values. The following example returns one of the inner functions from the outer parent() function:

In [7]:
def parent(num):
    def first_child():
        return "Hi, I am Emma"

    def second_child():
        return "Call me Liam"

    if num == 1:
        return first_child
    else:
        return second_child

Note that you are returning first_child without the parentheses. Recall that this means that you are returning a reference to the function first_child. In contrast first_child() with parentheses refers to the result of evaluating the function. This can be seen in the following example:

In [8]:
first = parent(1)

In [9]:
first

<function __main__.parent.<locals>.first_child()>

In [10]:
second = parent(2)

In [11]:
second

<function __main__.parent.<locals>.second_child()>

The somewhat cryptic output simply means that the first variable refers to the local first_child() function inside of parent(), while second points to second_child().

You can now use first and second as if they are regular functions, even though the functions they point to can’t be accessed directly:

In [12]:
first()

'Hi, I am Emma'

In [13]:
second()

'Call me Liam'

Finally, note that in the earlier example you executed the inner functions within the parent function, for instance first_child(). However, in this last example, you did not add parentheses to the inner functions—first_child—upon returning. That way, you got a reference to each function that you could call in the future. Make sense?

## Simple Decorators

Now that you’ve seen that functions are just like any other object in Python, you’re ready to move on and see the magical beast that is the Python decorator. Let’s start with an example:

In [14]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

In [15]:
def say_whee():
    print("Whee!")

In [16]:
say_whee = my_decorator(say_whee)

Can you guess what happens when you call say_whee()? Try it:

In [17]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


To understand what’s going on here, look back at the previous examples. We are literally just applying everything you have learned so far.

The so-called decoration happens at the following line:

In [18]:
say_whee = my_decorator(say_whee)

In effect, the name say_whee now points to the wrapper() inner function. Remember that you return wrapper as a function when you call my_decorator(say_whee):

In [19]:
say_whee

<function __main__.my_decorator.<locals>.wrapper()>

However, wrapper() has a reference to the original say_whee() as func, and calls that function between the two calls to print().

Put simply: decorators wrap a function, modifying its behavior.

Before moving on, let’s have a look at a second example. Because wrapper() is a regular Python function, the way a decorator modifies a function can change dynamically. So as not to disturb your neighbors, the following example will only run the decorated code during the day:

In [20]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep
    return wrapper

In [21]:
def say_whee():
    print("Whee!")

In [22]:
say_whee = not_during_the_night(say_whee)

If you try to call say_whee() after bedtime, nothing will happen:

In [23]:
say_whee()

Whee!


### Syntactic Sugar


The way you decorated say_whee() above is a little clunky. First of all, you end up typing the name say_whee three times. In addition, the decoration gets a bit hidden away below the definition of the function.

Instead, Python allows you to use decorators in a simpler way with the @ symbol, sometimes called the “pie” syntax. The following example does the exact same thing as the first decorator example:

In [1]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

In [2]:
@my_decorator
def say_whee():
    print("Whee!")

In [3]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


In [5]:
def say_whee():
    print("Whee!")

In [6]:
say_whee()

Whee!


So, @my_decorator is just an easier way of saying say_whee = my_decorator(say_whee). It’s how you apply a decorator to a function.

### Reusing Decorators

Recall that a decorator is just a regular Python function. All the usual tools for easy reusability are available. Let’s move the decorator to its own module that can be used in many other functions.

Create a file called decorators.py with the following content:

In [7]:
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

> Note: You can name your inner function whatever you want, and a generic name like wrapper() is usually okay. You’ll see a lot of decorators in this article. To keep them apart, we’ll name the inner function with the same name as the decorator but with a wrapper_ prefix.

In [ ]:
from decorators import do_twice

@do_twice
def say_whee():
    print("Whee!")

When you run this example, you should see that the original say_whee() is executed twice:



In [ ]:
say_whee()

### Decorating Functions With Arguments

Say that you have a function that accepts some arguments. Can you still decorate it? Let’s try:



In [ ]:
from decorators import do_twice

@do_twice
def greet(name):
    print(f"Hello {name}")

Unfortunately, running this code raises an error:

In [ ]:
 greet("World")

The problem is that the inner function wrapper_do_twice() does not take any arguments, but name="World" was passed to it. You could fix this by letting wrapper_do_twice() accept one argument, but then it would not work for the say_whee() function you created earlier.

The solution is to use `*args` and ` **kwargs`  in the inner wrapper function. Then it will accept an arbitrary number of positional and keyword arguments. Rewrite decorators.py as follows:

In [ ]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

The wrapper_do_twice() inner function now accepts any number of arguments and passes them on to the function it decorates. Now both your say_whee() and greet() examples works:

### Returning Values From Decorated Functions

What happens to the return value of decorated functions? Well, that’s up to the decorator to decide. Let’s say you decorate a simple function as follows:

In [ ]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}

In [ ]:
hi_adam = return_greeting("Adam")

In [ ]:
print(hi_adam)

Oops, your decorator ate the return value from the function.

Because the do_twice_wrapper() doesn’t explicitly return a value, the call return_greeting("Adam") ended up returning None.

To fix this, you need to make sure the wrapper function returns the return value of the decorated function. Change your decorators.py file:

In [ ]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

The return value from the last execution of the function is returned:

In [ ]:
return_greeting("Adam")

### Who Are You, Really?

A great convenience when working with Python, especially in the interactive shell, is its powerful introspection ability. Introspection is the ability of an object to know about its own attributes at runtime. For instance, a function knows its own name and documentation:

In [27]:
print

<function print>

In [28]:
print.__name__

'print'

In [29]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



The introspection works for functions you define yourself as well:



In [30]:
say_whee

<function __main__.my_decorator.<locals>.wrapper()>

In [31]:
say_whee.__name__

'wrapper'

In [32]:
help(say_whee)

Help on function wrapper in module __main__:

wrapper()



However, after being decorated, say_whee() has gotten very confused about its identity. It now reports being the wrapper_do_twice() inner function inside the do_twice() decorator. Although technically true, this is not very useful information.

To fix this, decorators should use the @functools.wraps decorator, which will preserve information about the original function. Update decorators.py again:

In [33]:
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

You do not need to change anything about the decorated say_whee() function:

In [37]:
@do_twice
def say_whee():
    print("Whee!")

In [38]:
say_whee

<function __main__.say_whee()>

In [39]:
say_whee.__name__

'say_whee'

In [40]:
help(say_whee)

Help on function say_whee in module __main__:

say_whee()



Much better! Now say_whee() is still itself after decoration.

> Technical Detail: The @functools.wraps decorator uses the function functools.update_wrapper() to update special attributes like __name__ and __doc__ that are used in the introspection.

## A Few Real World Examples

### Primer 1: Timing

In [25]:
# povemo da bomo to pogleali pri benchmarkingu
import functools
import time

def logtime(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time() 
        result = func(*args, **kwargs)
        total_time = time.time() - start_time
        
        with open('timelog.txt', 'a') as outfile:
            outfile.write(f'[{time.time()}]\t{func.__name__}\t{total_time}\n')
            
        return result
    return wrapper

@logtime
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [26]:
waste_some_time(10)

In [27]:
!cat timelog.txt

[1586514172.133054]	waste_some_time	0.06323385238647461
[1586514184.569715]	waste_some_time	0.06529951095581055


In [56]:
!rm timelog.txt

rm: cannot remove 'timelog.txt': No such file or directory


### Primer 2: Runs once per minute

In [31]:
import functools
import time

class CalledTooOftenError(Exception):
    pass

def once_per_minute(func):
    last_invoked = 0
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        nonlocal last_invoked
        elapsed_time = time.time() - last_invoked
        if elapsed_time < 60:
            raise CalledTooOftenError(f'Only {elapsed_time} has passed')
        last_invoked = time.time()
        return func(*args, **kwargs)
    return wrapper

In [32]:
@once_per_minute
def add(a,b):
    return a+ b

In [33]:
print(add(2,2))

4


In [34]:
print(add(2,3))

CalledTooOftenError: Only 5.383094310760498 has passed

### Primer 3: Runs once per n seconds

In [35]:
import functools
import time

class CalledTooOftenError(Exception):
    pass

def once_per_n(n): # excetute once when we get an argument
    def middle(func):
        last_invoked = 0
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            nonlocal last_invoked
            elapsed_time = time.time() - last_invoked
            if elapsed_time < n:
                raise CalledTooOftenError(f'Only {elapsed_time} has passed')
            last_invoked = time.time()
            return func(*args, **kwargs)
        return wrapper
    return middle

In [36]:
@once_per_n(5)
def add(a,b):
    return a+ b

In [40]:
print(add(2,2))

4


In [42]:
print(add(2,3))

5


### Primer 4: Memoization

In [50]:
import functools
import pickle

def memoize(func): 
    cache = {} #zažene se ko dekoriramo funkcijo
    @functools.wraps(func)
    def wrapper(*args, **kwargs): # zažene se vsakič ko je dekororana funkcija zagnana
        t = (pickle.dumps(args), pickle.dumps(kwargs))
        if t not in cache:
            print(f'Caching NEW value for {func.__name__}{args}')
            cache[t] = func(*args, **kwargs)
        else:
            print(f'Using OLD value for {func.__name__}{args}')
        return cache[t]
    return wrapper

In [51]:
@memoize
def add(a,b):
    print(f'funkcija add zagnana')
    return a+ b

In [52]:
print(add(2,2))

Caching NEW value for add(2, 2)
4


In [53]:
print(add(2,2))

Using OLD value for add(2, 2)
4


In [54]:
print(add(2,3))

Caching NEW value for add(2, 3)
5


In [55]:
print(add(2,2))

Using OLD value for add(2, 2)
4


### Primer 5: Debugging Code

The following @debug decorator will print the arguments a function is called with as well as its return value every time the function is called:

In [44]:
import functools

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")           # 4
        return value
    return wrapper_debug

The signature is created by joining the string representations of all the arguments. The numbers in the following list correspond to the numbered comments in the code:
- Create a list of the positional arguments. Use repr() to get a nice string representing each argument.
- Create a list of the keyword arguments. The f-string formats each argument as key=value where the !r specifier means that repr() is used to represent the value.
- The lists of positional and keyword arguments is joined together to one signature string with each argument separated by a comma.
- The return value is printed after the function is executed.

Let’s see how the decorator works in practice by applying it to a simple function with one position and one keyword argument:

The following example calculates an approximation to the mathematical constant e:



In [48]:
import math

# Apply a decorator to a standard library function
math.factorial = debug(math.factorial)

In [49]:
def approximate_e(terms=18):
    return sum(1 / math.factorial(n) for n in range(terms))

This example also shows how you can apply a decorator to a function that has already been defined. 

When calling the approximate_e() function, you can see the @debug decorator at work:

In [50]:
approximate_e(5)

Calling factorial(0)
'factorial' returned 1
Calling factorial(1)
'factorial' returned 1
Calling factorial(2)
'factorial' returned 2
Calling factorial(3)
'factorial' returned 6
Calling factorial(4)
'factorial' returned 24


2.708333333333333

In this example, you get a decent approximation to the true value e = 2.718281828, adding only 5 terms.

### Primer 6: Attributes

Give many objects the same attributes, but without using inheritance.

The classes aren't related so no inheritance.

Let's improve `__repr__`

In [57]:
def fancy_repr(self):
    return f"I'm a {type(self)}, with vars {vars(self)}"

In [63]:
def better_repr(c): # decorator class
    c.__repr__ = fancy_repr
    @functools.wraps(c)
    def wrapper(*args, **kwargs):
        o = c(*args, **kwargs)
        o._created_at = time.time()
        return o
    return wrapper

In [64]:
@better_repr
class Foo():
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
f = Foo(10, [10,20,30])
print(f)

I'm a <class '__main__.Foo'>, with vars {'x': 10, 'y': [10, 20, 30], '_created_at': 1586533757.899802}


In [65]:
f._created_at

1586533757.899802

[Primeri decoratorjev](https://wiki.python.org/moin/PythonDecoratorLibrary)

## Vaje

### Vaja 1

In [97]:
# brez dekoratorja
def read_logs(path):
    with open(path, 'r') as f:
        file = f.readlines()
    return file

In [98]:
logs = read_logs('data/decorators_text.txt')

In [99]:
logs

['2006-09-19 03:26:26 DROP UDP 172.20.73.241 239.255.255.250 2250 1900 315 - - - - - - - RECEIVE\n',
 '2006-09-19 03:26:26 DROP UDP 172.20.73.241 239.255.255.250 2250 1900 305 - - - - - - - RECEIVE\n',
 '2006-09-19 03:27:05 CLOSE UDP 192.168.72.12 10.20.72.186 3682 88 - - - - - - - - -\n',
 '2006-09-19 03:27:05 CLOSE UDP 192.168.72.12 10.20.72.186 3683 88 - - - - - - - - -\n',
 '2006-09-19 03:27:53 CLOSE TCP 192.168.72.12 10.20.72.186 3680 445 - - - - - - - - -\n',
 '2006-09-19 03:29:24 DROP UDP 172.20.73.241 239.255.255.250 2250 1900 250 - - - - - - - RECEIVE\n',
 '2006-09-19 03:29:24 DROP UDP 172.20.73.241 239.255.255.250 2250 1900 315 - - - - - - - RECEIVE\n',
 '2006-09-19 03:29:24 DROP UDP 172.20.73.241 239.255.255.250 2250 1900 263 - - - - - - - RECEIVE\n',
 '2006-09-19 03:29:24 DROP UDP 172.20.73.241 239.255.255.250 2250 1900 315 - - - - - - - RECEIVE\n',
 '2006-09-19 03:29:24 DROP UDP 172.20.73.241 239.255.255.250 2250 1900 305 - - - - - - - RECEIVE\n',
 '2006-09-19 03:31:15 DRO

In [93]:
def remove_public_ips(func):
    @functools.wraps(func)
    def wrapper_remove_public_ips(*args, **kwargs):
        text = func(*args, **kwargs)
        clean_text = []
        for line in text:
            line_splited = line.split()
            src_ip = line_splited[5]
            if not (src_ip.startswith('192.168') or src_ip.startswith('10.')):
                line_splited[5] = 'XXX.XXX.XXX.XXX'
                clean_line = ' '.join(line_splited)
                clean_text.append(clean_line)
            else:
                clean_text.append(line)
        return clean_text
    return wrapper_remove_public_ips

In [94]:
@remove_public_ips
def read_logs(path):
    with open(path, 'r') as f:
        file = f.readlines()
    return file

In [95]:
logs = read_logs('data/decorators_text.txt')

In [96]:
logs

['2006-09-19 03:26:26 DROP UDP 172.20.73.241 XXX.XXX.XXX.XXX 2250 1900 315 - - - - - - - RECEIVE',
 '2006-09-19 03:26:26 DROP UDP 172.20.73.241 XXX.XXX.XXX.XXX 2250 1900 305 - - - - - - - RECEIVE',
 '2006-09-19 03:27:05 CLOSE UDP 192.168.72.12 10.20.72.186 3682 88 - - - - - - - - -\n',
 '2006-09-19 03:27:05 CLOSE UDP 192.168.72.12 10.20.72.186 3683 88 - - - - - - - - -\n',
 '2006-09-19 03:27:53 CLOSE TCP 192.168.72.12 10.20.72.186 3680 445 - - - - - - - - -\n',
 '2006-09-19 03:29:24 DROP UDP 172.20.73.241 XXX.XXX.XXX.XXX 2250 1900 250 - - - - - - - RECEIVE',
 '2006-09-19 03:29:24 DROP UDP 172.20.73.241 XXX.XXX.XXX.XXX 2250 1900 315 - - - - - - - RECEIVE',
 '2006-09-19 03:29:24 DROP UDP 172.20.73.241 XXX.XXX.XXX.XXX 2250 1900 263 - - - - - - - RECEIVE',
 '2006-09-19 03:29:24 DROP UDP 172.20.73.241 XXX.XXX.XXX.XXX 2250 1900 315 - - - - - - - RECEIVE',
 '2006-09-19 03:29:24 DROP UDP 172.20.73.241 XXX.XXX.XXX.XXX 2250 1900 305 - - - - - - - RECEIVE',
 '2006-09-19 03:31:15 DROP UDP 192.168.